# Model Evaluation
This model is tested on a much reliable test dataset.

Credits to all AA ASL 2018 members who helped in annotating the data!

In [14]:
import pandas as pd
import numpy as np
from fastai import *
from fastai.text import *

In [15]:
df = pd.read_csv('../data/human_label.csv')

In [ ]:
df.describe()

In [ ]:
df['Human Label'].value_counts().plot('bar')

In [18]:
df.rename(index=str, columns={"Human Label": "label"}, inplace=True)
df.rename(index=str, columns={"cleansed_comment": "text"}, inplace=True)

In [19]:
df = df[df.label != 'Neutral'] # filter out Neutral, it is not in our model
df.label = df.label.apply(lambda x: x.lower())
df.reset_index(drop=True, inplace=True)

In [20]:
path = Path('/Users/jiajunkoh/projects/aa-nps')

In [21]:
learn = load_learner(path, 'models/awd_lstm_classifier_model.pkl')

In [22]:
COLS = [
    'text',
    'label',
    'prob_neg',
    'prob_pos',
]

In [23]:
# @TODO: Wrap this into batch predict
correct_label, wrong_label = 0, 0
metrics = {'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0}
mis_clas = []
with open('../metrics/metrics.csv', 'w') as f:
    f.write(','.join(COLS))
    f.write('\n')
    for ind, row in df.iterrows():
        if (ind % 100 == 0):
            print(ind, df.shape[0]) # 'progress bar' lul
        p = learn.predict(row.text)
        actual = 1 if row.label == 'positive' else 0 # it's easier to deal with number
        if p[1] == actual:
            correct_label += 1
            if p[1] == 1:
                metrics['tp'] += 1
            else:
                metrics['tn'] += 1
        else:
            wrong_label += 1
            # lets look into the mis-classified items
            mis_clas.append([row.text, p[2][0].item(), p[2][1].item()])
            if p[1] == 1:
                metrics['fp'] += 1
            else:
                metrics['fn'] += 1
        # NOTE 
        # !! p[2][0] - Neg, p[2][1] - Pos !!
        csv_row = [row.text, row.label, str(p[2][0].item()), str(p[2][1].item())]
        f.write(','.join(csv_row))
        f.write('\n')

0 1557
100 1557
200 1557
300 1557
400 1557
500 1557
600 1557
700 1557
800 1557
900 1557
1000 1557
1100 1557
1200 1557
1300 1557
1400 1557
1500 1557


In [24]:
correct_label, wrong_label

(1508, 49)

In [25]:
accuracy = (metrics['tp'] + metrics['tn'])/(metrics['tp'] + metrics['fp'] + metrics['tn'] + metrics['fn'])
precision = metrics['tp']/(metrics['tp'] + metrics['fp'])
recall = metrics['tp']/(metrics['tp'] + metrics['fn'])
f1 = 2*precision*recall/(precision + recall)

accuracy, precision, recall, f1

(0.968529222864483, 0.9804772234273319, 0.9186991869918699, 0.9485834207764953)

In [ ]:
mis_clas